# Parte 2 - Sistema de Recomendação  

Vamos utilizar o gênero, o ano de lançamento e o rating para sugerir filme similares a outros bem avaliados pelo usuário.

In [393]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [394]:
import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

In [395]:
data = pd.read_csv(r'movie_dataset.csv')

In [396]:
data.head()

,movieId,title,release_date,gen:(no genres listed),gen:Action,gen:Adventure,gen:Animation,gen:Children,gen:Comedy,gen:Crime,...,gen:Sci-Fi,gen:Thriller,gen:War,gen:Western,num_gen,userId,rating,hour,mean_rat_user,mean_rat_movie
0,1,Toy Story (1995),1995.0,0,0,1,1,1,1,0,...,0,0,0,0,5,3,4.0,13,4.122995,3.970998
1,1,Toy Story (1995),1995.0,0,0,1,1,1,1,0,...,0,0,0,0,5,6,5.0,17,3.750000,3.970998
2,1,Toy Story (1995),1995.0,0,0,1,1,1,1,0,...,0,0,0,0,5,8,4.0,13,3.800000,3.970998
3,1,Toy Story (1995),1995.0,0,0,1,1,1,1,0,...,0,0,0,0,5,10,4.0,2,3.894737,3.970998
4,1,Toy Story (1995),1995.0,0,0,1,1,1,1,0,...,0,0,0,0,5,11,4.5,1,3.945437,3.970998


In [397]:
data.shape

(422364, 29)

In [398]:
X = data.drop_duplicates(subset='movieId')
X = X.reset_index(drop=True)
X.head()

,movieId,title,release_date,gen:(no genres listed),gen:Action,gen:Adventure,gen:Animation,gen:Children,gen:Comedy,gen:Crime,...,gen:Sci-Fi,gen:Thriller,gen:War,gen:Western,num_gen,userId,rating,hour,mean_rat_user,mean_rat_movie
0,1,Toy Story (1995),1995.0,0,0,1,1,1,1,0,...,0,0,0,0,5,3,4.0,13,4.122995,3.970998
1,2,Jumanji (1995),1995.0,0,0,1,0,1,0,0,...,0,0,0,0,3,1,3.5,23,3.742857,3.287346
2,3,Grumpier Old Men (1995),1995.0,0,0,0,0,0,1,0,...,0,0,0,0,2,2,4.0,15,4.000000,3.175926
3,4,Waiting to Exhale (1995),1995.0,0,0,0,0,0,1,0,...,0,0,0,0,3,41,2.0,23,4.118644,3.110169
4,5,Father of the Bride Part II (1995),1995.0,0,0,0,0,0,1,0,...,0,0,0,0,1,12,2.0,20,3.527778,3.154762


In [399]:
X2 = X.copy()

In [400]:
X = X.drop(['movieId','title','hour','num_gen','userId','rating','mean_rat_user'],axis=1)
X.head(10)

,release_date,gen:(no genres listed),gen:Action,gen:Adventure,gen:Animation,gen:Children,gen:Comedy,gen:Crime,gen:Documentary,gen:Drama,...,gen:Horror,gen:IMAX,gen:Musical,gen:Mystery,gen:Romance,gen:Sci-Fi,gen:Thriller,gen:War,gen:Western,mean_rat_movie
0,1995.0,0,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.970998
1,1995.0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.287346
2,1995.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,3.175926
3,1995.0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,3.110169
4,1995.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.154762
5,1995.0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,3.833333
6,1995.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,3.393519
7,1995.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.093458
8,1995.0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.423423
9,1995.0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,3.718468


In [401]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
X = MinMaxScaler().fit_transform(X)
#X_train, X_test = train_test_split(X, test_size=0.5, random_state=42)

In [402]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=4,algorithm='ball_tree').fit(X)

In [403]:
distances, indices = knn.kneighbors(X,n_neighbors=10)

In [404]:
# colocar a função print_similar_movie pra funcionar a partir da função get_id_from_partial_title
def get_index_from_title(title):
    return X2[X2['title']==title].index.tolist()[0]

In [405]:
# set é usado pra listar apenas itens não repetidos
all_movie_titles = list(set(data['title'].values))

def get_id_from_partial_title(partial):
    partial = partial.lower()
    for title in all_movie_titles:
        if partial in title.lower():
            print(title,all_movie_titles.index(title))

In [406]:
def print_similar_movie(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(X2.ix[id]['title'])
     
    if query:
        found_id = get_index_from_title(query)
        for id in indices[found_id][1:]:
            print(X2.ix[id]['title'])

In [407]:
# recomenda três filmes para cada um dos gêneros favoritos do usuário

''' lembrar de considerar os casos em que não há filme disponível'''

def recom_movie(userId):
    gen_list = gen_fav(userId)
    for item in gen_list:
        id = X2[X2['userId']==userId & X2[item]].sort_values(['rating'],ascending=[False])['movieId'].index[0]
        print('Baseado no seu gosto por',item,'veja:\n')
        for id in indices[id][1:]:
            print(X2.ix[id]['title'])
        print('')

In [408]:
# identifica os três gêneros favoritos do usuário
def gen_fav(userId):
    gen_cols = ['gen:(no genres listed)','gen:Action','gen:Adventure','gen:Animation','gen:Children','gen:Comedy','gen:Crime',
                'gen:Documentary','gen:Drama','gen:Fantasy','gen:Film-Noir','gen:Horror','gen:IMAX','gen:Musical','gen:Mystery',
                'gen:Romance', 'gen:Sci-Fi','gen:Thriller','gen:War','gen:Western']
    gen_corr = X2[X2['userId']==userId].corr(method='pearson')
    return gen_corr['rating'][gen_cols].sort_values(ascending=False).index[:3]

In [409]:
recom_movie(1)

Baseado no seu gosto por gen:Fantasy veja:

Lord of the Rings: The Two Towers, The (2002)
Harry Potter and the Chamber of Secrets (2002)
Dark Crystal, The (1982)
Excalibur (1981)
Spirited Away (Sen to Chihiro no kamikakushi) (2001)
Fall, The (2006)
Life Aquatic with Steve Zissou, The (2004)
Harry Potter and the Prisoner of Azkaban (2004)
Hobbit: An Unexpected Journey, The (2012)

Baseado no seu gosto por gen:Adventure veja:

Lord of the Rings: The Fellowship of the Ring, The (2001)
Harry Potter and the Chamber of Secrets (2002)
Dark Crystal, The (1982)
Excalibur (1981)
Spirited Away (Sen to Chihiro no kamikakushi) (2001)
Fall, The (2006)
Life Aquatic with Steve Zissou, The (2004)
Harry Potter and the Prisoner of Azkaban (2004)
Hobbit: An Unexpected Journey, The (2012)

Baseado no seu gosto por gen:Romance veja:

Kingdom of Heaven (2005)
Pearl Harbor (2001)
Henry V (1989)
Spartacus (1960)
English Patient, The (1996)
Patriot, The (2000)
Thin Red Line, The (1998)
Behind Enemy Lines (2001)

# Parte 3 - Avaliação do sistema de recomendação

Para avaliar o sistema de recomendação contruído não cabem as métricas usuais de classificação (accuracy, precision, F1, Recall e Confusion Matrix) e regressão (R2, RMSE, MAE e Explained Variance). De qualquer forma precisamos de uma forma de avaliá-lo.

Uma maneira seria fazer um teste AB: se um grupo de usuários avaliar positivamente os filmes sugeridos teríamos um indicativo de sucesso nas sugestões, mas não há como fazer isso durante esse problema. Vamos então utilizar uma métrica qualitativa e discutir sobre outra quantitativa.

### Métrica qualitativa

Vamos criar uma lista de filmes favoritos de determinado gênero de um usuário bem como uma lista de sugestões. Desta forma verificamos se algum dos filmes sugeridos entá na lista de favoritos, cada sugestão que atender esta especificação será considerada um sucesso. Por fim, contamos o número de sucessos totais depois de *n* recomendações para *m* usuários. Chamamos de métrica qualitativa pois não se encaixa em um padrão bem estabelecido e o tamanho das listar é não-definido. Se a lista de favoritos for grande, da ordem do número de filmes avaliados, invariavelmente obteremos sucesso na recomendação.

A partir disso poderíamos criar uma maneira de utilizar as métricas de classificação como segue abaixo:

| Classificação | Condição      | 
| ------------- |:-------------:|
| TP            | $A \in B$     |
| FP            | $A \notin B$  |
| TN            | $X \setminus{A} \notin B$|
| FN            | $X \setminus{A} \notin B$|

onde A são os filmes sugeridos, B os filmes favoritos e X todos os filmes. Para não fugir do objetivo, vamos usar apenas a ideia dos sucessos.

### Métrica quantitativa

Dadas as listas de filmes favoritos e sugeridos, é possível utilizar o método *kneighbors* para obter as distâncias entre dois datapoints ou a distância média entre as listas. Nesse caso quanto menor a distância média, melhores são as sugestões.


Sugestão de referência: https://frnsys.com/ai_notes/machine_learning/unsupervised_learning.html

### Recomendando

In [410]:
# fornece uma lista 3 filmes recomendados ao usuário
def recom_movie_brute(userId):
    movie_list = []
    gen = gen_fav(userId)[0]
    if len(X2[X2['userId']==userId & X2[gen]].sort_values(['rating'],ascending=[False])['movieId']) ==0:
        return []
    else:
        id = X2[X2['userId']==userId & X2[gen]].sort_values(['rating'],ascending=[False])['movieId'].index[0]
        for id in indices[id][1:]:
            movie_list.append(X2.ix[id]['movieId'])
        return movie_list

In [411]:
# acabei não usando

# lista de filmes avaliados pelo usuário
rated_movies = X2['movieId'][X2['userId']==userId].unique()

In [412]:
# fornece uma lista 3 filmes recomendados ao usuário para cada um dos 3 gêneros favoritos
def check_rec(userId):
    if recom_movie_brute(userId) == None:
        return 'Nenhuma indicação'
    else:
        rec_list = recom_movie_brute(userId)
        real_list = list(X2[X2['userId']==userId & X2[gen_fav(userId)[2]]].sort_values(['rating'],ascending=[False])['movieId'].iloc[:5])

        for i in rec_list:
            if i in real_list:
                print('Ok')

### Sucesso

In [413]:
def overall_sucess():
    sucess=0
    for userId in range(1,100):
        user = X2[X2['userId']==userId & X2[gen_fav(userId)[0]]].sort_values(['rating'],ascending=[False])
        sucess = sucess + user['movieId'].iloc[:10].isin(recom_movie_brute(userId)).sum()
    return sucess

In [414]:
overall_sucess()

7

## Fine-tunning

Para melhorar o desempenho das recomendações vamos usar como parâmetro o número de sucessos nos primeiros 100 usuários sugerindo 10 filmes para cada um e a partir disso modificar os parâmetros do algorimo:

In [415]:
results = pd.DataFrame({'neighbors':[],'algorithm':[],'leaf_size':[],'sucess':[]})
for neigh in [4,7,10,15]:
    for alg in ['auto', 'ball_tree','kd_tree', 'brute']:
        for n_leaves in [20,30,40]:
            knn = NearestNeighbors(n_neighbors=neigh,algorithm=alg,leaf_size=n_leaves).fit(X)
            distances, indices = knn.kneighbors(X,n_neighbors=20)
            new_result = pd.DataFrame({'neighbors':[neigh],'algorithm':[alg],'leaf_size':[n_leaves],'sucess':[overall_sucess()]})
            results = pd.concat([results,new_result],axis=0)
results.sort_values(by='sucess',ascending=False).head(15)

,neighbors,algorithm,leaf_size,sucess
0,4.0,auto,20.0,14.0
0,4.0,auto,30.0,14.0
0,10.0,auto,40.0,14.0
0,10.0,ball_tree,20.0,14.0
0,10.0,ball_tree,30.0,14.0
0,10.0,ball_tree,40.0,14.0
0,10.0,kd_tree,20.0,14.0
0,10.0,kd_tree,30.0,14.0
0,10.0,kd_tree,40.0,14.0
0,10.0,brute,20.0,14.0


In [416]:
results = pd.DataFrame({'neighbors':[],'algorithm':[],'leaf_size':[],'metric':[],'sucess':[]})
for neigh in [2,7,10,15]:
    for alg in ['auto']:
        for n_leaves in [20,30,40]:
            for metrics in ['cityblock','cosine','euclidean','l1','l2','manhattan']:
                knn = NearestNeighbors(n_neighbors=neigh,algorithm=alg,leaf_size=n_leaves,metric=metrics).fit(X)
                distances, indices = knn.kneighbors(X,n_neighbors=5)
                new_result = pd.DataFrame({'neighbors':[neigh],'algorithm':[alg],'leaf_size':[n_leaves],'metric':[metrics],'sucess':[overall_sucess()]})
                results = pd.concat([results,new_result],axis=0)
results.sort_values(by='sucess',ascending=False)  

,neighbors,algorithm,leaf_size,metric,sucess
0,2.0,auto,20.0,cityblock,3.0
0,2.0,auto,20.0,cosine,3.0
0,10.0,auto,40.0,l2,3.0
0,10.0,auto,40.0,l1,3.0
0,10.0,auto,40.0,euclidean,3.0
0,10.0,auto,40.0,cosine,3.0
0,10.0,auto,40.0,cityblock,3.0
0,10.0,auto,30.0,manhattan,3.0
0,10.0,auto,30.0,l2,3.0
0,10.0,auto,30.0,l1,3.0


Some distances that you can use include Euclidean distance, Manhattan distance (also known as the city block distance or the taxicab distance), Minkowski distance (a generalization of the Manhattan and Euclidean distances), and Mahalanobis distance.

# Conclusões

Vamos discutir vários aspectos do sistema de recomendação construído abordando suas falhas e possíveis melhorias.

### Dados

#### Sobre as features disponíveis

O link sugerido para acessar o dataset leva ao dataset chamado ml-20m e neste não há informações mais específicas sobre os usuários. Há um dataset menor, ml-100k, na mesma base que possui os dados de idade, gênero, profissão e endereço (zipcode) do usuário. Caso estas informações fossem utilizadas poderíamos encontrar filmes favoritos do usuário com um algoritmo, como o NearestNeighbors, para identificar filmes favoritos e utilizá-los como referência para filmes similares.

Outra característica que poderia ser utilizada neste algoritmo seria o 'timestamp', pois filmes vistos por último e com boa avaliação podem ser uma sugestão melhor que filmes vistos há muito tempo com boa avaliação.

Como o dataset ml-100k não foi indicado primariamente vamos assumir que não temos estes dados e utilizar a correlação de Pearson e entre o 'rating' e os gêneros para determinar os gêneros favoritos e utilizá-los para fazer a sugestão.

Aqui foram usadas variáveis pouco significativas para classificar um filme. Com mais informações como o país de origem, duração, língua, protagonistas e avaliação de sites como Rotten Tomatoes e IMDB enriqueceríamos a capacidade de encontrar semelhanças entre os filmes. Estes dados podem ser encontrados com a própria base de dados do IMDB, mas vamos focar no simples aqui.

### Métricas para ranqueamento

Um sistema com avaliação do tipo "like" e "dislike" permitiria a construção de uma modelo de classificação

### Modelo

#### Regressão

#### Classificação

#### Não supervisionado


### Avaliação do modelo